In [41]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ast import literal_eval
import os, sys
sys.path.append('../pubmed_rag')
#from utils import get_chunks
from bioc import (
    mean_pooling
)


get the data for the vx db first. 

Note: The primary key and vector fields use their default names (“id” and “vector”).

to insert into vx database need each row as a dictionary

In [36]:
# read in
filename = '/Users/anglup/GitHub/pubmed-rag/biocjson/tester.csv'
df = pd.read_csv(filename, )
# rename id and vector cols
df = df.rename(
    columns={
        'Unnamed: 0':'id',
        'embedding':'vector'
    }, 
)
# ensure vector is a list
df['vector'] = df['vector'].apply(literal_eval)

# sanity check
#data.head()

# to list of dicts
data = df.to_dict(orient='records')

In [44]:
df.head()

,id,section,pmid,pmcid,date,authors,journal,text,vector
0,0,abstract,28924171,PMC5603535,2017-09-18 00:00:00+00:00,Luo Y and Zhao X and Zhou J and Yang J and Zha...,Nat Commun,"The emergence of large-scale genomic, chemical...","[-0.027584053575992584, 0.054618917405605316, ..."
1,1,discuss,28924171,PMC5603535,2017-09-18 00:00:00+00:00,Luo Y and Zhao X and Zhou J and Yang J and Zha...,Nat Commun,Discussion . Recent advances in large-scale ex...,"[0.00014715130964759737, 0.054599590599536896,..."
2,2,discuss,28924171,PMC5603535,2017-09-18 00:00:00+00:00,Luo Y and Zhao X and Zhou J and Yang J and Zha...,Nat Commun,"On the other hand, based on the systematic int...","[0.00014715130964759737, 0.054599590599536896,..."
3,3,discuss,28924171,PMC5603535,2017-09-18 00:00:00+00:00,Luo Y and Zhao X and Zhou J and Yang J and Zha...,Nat Commun,Our findings are particularly helpful for unde...,"[0.00014715130964759737, 0.054599590599536896,..."
4,4,intro,28924171,PMC5603535,2017-09-18 00:00:00+00:00,Luo Y and Zhao X and Zhou J and Yang J and Zha...,Nat Commun,Introduction . Computational prediction of dru...,"[-0.039880603551864624, -0.011334901675581932,..."


In [2]:
from pymilvus import MilvusClient

In [37]:
client = MilvusClient('demo.db')

if client.has_collection(collection_name="demo_collection"):
    client.drop_collection(collection_name="demo_collection")
client.create_collection(
    collection_name="demo_collection",
    dimension=768,  # The vectors we will use in this demo has 768 dimensions
)


In [39]:
result = client.insert(collection_name="demo_collection", data=data)
print(result)


{'insert_count': 47, 'ids': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46]}


yay first vx db created

In [40]:
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F

chosen_model = 'sentence-transformers/all-mpnet-base-v2'

#logger.info(f"Loading model {chosen_model} from HuggingFace")
tokenizer = AutoTokenizer.from_pretrained(chosen_model)
model = AutoModel.from_pretrained(chosen_model)

/Users/anglup/Library/Caches/pypoetry/virtualenvs/pubmed-rag-iB5Z1S4t-py3.11/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [75]:
# Tokenize sentences
encoded_input = tokenizer(
    "what knowledge graph can tell me clinical information", 
    padding=True, 
    truncation=False, 
    return_tensors='pt'
)

# Compute token embeddings
with torch.no_grad():
    model_output = model(**encoded_input)

# Perform pooling
sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

# Normalize embeddings
sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)

print("Sentence embeddings:")
print(sentence_embeddings)


Sentence embeddings:
tensor([[ 1.1717e-02, -9.0640e-03, -8.6910e-03,  2.9574e-03, -6.6460e-03,
          2.0472e-03, -1.1603e-02,  7.7256e-02,  8.0103e-02,  3.7327e-02,
          5.4018e-02, -1.2791e-02,  9.2674e-03,  7.3003e-02, -4.7422e-03,
         -2.7896e-02, -3.1326e-03,  3.2666e-02,  5.4093e-02,  5.6401e-03,
         -2.7114e-02,  9.1211e-03, -4.6526e-02,  2.5128e-02, -2.2649e-02,
          9.8743e-03,  1.9677e-02,  9.5495e-03,  1.0500e-02, -7.8114e-02,
         -2.3350e-02, -2.3098e-02, -5.6125e-02, -5.8920e-02,  1.2445e-06,
         -3.7598e-02, -7.3195e-03,  2.0765e-02, -2.9545e-02, -7.4683e-02,
          1.7828e-02, -5.6797e-02,  5.1238e-02, -2.8561e-02, -8.2972e-03,
         -3.1635e-02,  3.6280e-02,  2.8717e-03,  5.3786e-03,  2.2265e-02,
          6.5829e-03,  3.5095e-02, -1.3864e-02, -4.9821e-03,  6.8762e-03,
         -2.1955e-02,  5.0770e-02,  3.0422e-02,  4.7645e-02, -4.1595e-02,
         -3.1964e-02,  3.6310e-02, -5.6259e-02,  5.9862e-03,  4.4858e-02,
         -2.6410e

In [76]:
result = client.search(
    collection_name="demo_collection",
    data=sentence_embeddings.detach().numpy(),
    filter="section == 'abstract'",
    limit=2,
    output_fields=["text", "pmid", 'section'],
)


In [78]:
result[0][1]['entity']['text']

'The emergence of large-scale genomic, chemical and pharmacological data provides new opportunities for drug discovery and repositioning . In this work, we develop a computational pipeline, called DTINet, to predict novel drug-target interactions from a constructed heterogeneous network, which integrates diverse drug-related information . DTINet focuses on learning a low-dimensional vector representation of features, which accurately explains the topological properties of individual nodes in the heterogeneous network, and then makes prediction based on these representations via a vector space projection scheme . DTINet achieves substantial performance improvement over other state-of-the-art methods for drug-target interaction prediction . Moreover, we experimentally validate the novel interactions between three drugs and the cyclooxygenase proteins predicted by DTINet, and demonstrate the new potential applications of these identified cyclooxygenase inhibitors in preventing inflammator